# Column
id, title, author, genre, story, platform, age, rate, complete, thumbnail, website
    - id: 인덱스
    - title: 제목
    - author: 작가
    - genre: 장르
    - story: 줄거리
    - platform: 플랫폼 (네이버, 다음, 카카오, 레진코믹스 중 1)
    - age: 연령대
    - rate: 평점
    - free: 유료 여부 (0: 유료, 1: 무료)
    - complete: 완결여부
    - thumbnail: 썸네일
    - website: 웹 주소

## 완결 웹툰

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep

import pandas as pd
import re

url = 'http://webtoon.daum.net/'
pay = ['pay', 'free']
pay_match = '#sort=&genre=&tab=finish&day=tue'

# 로그인 정보
email = 'gunsong2@kakao.com'
pw = 'kvo20175607!'

title = []
total_tc = 0
now_tc = 0 
fr_ = 0

author = {}
genre = {}
story = {}
age = {}
rate = {}
free = {}
complete = {}
thumbnail = {}
website = {}

for p_ in pay:
    link = url + pay_match[0:6] + p_ + pay_match[6:] 
    
    # webdriver_manager를 사용해 execution_path 변수를 설정
    # 스크립트를 실행할 때 자동으로 드라이버 다운로드
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(link)
    
    # 다 뜰때까지 존버
    WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'tit_wt')))
    #WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'ico_comm ico_adult')))
    WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'txt_info')))
    WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'img_thumb')))
    WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'link_wt')))
    
    
    # 개발자도구에서 보이는 source code 가져옴
    html = driver.page_source
    bsObject = BeautifulSoup(html, 'html.parser')
    #print(bsObject)
    
    # title, age
    titles = bsObject.find_all('strong', {'class':'tit_wt'}) # find_all('태그명', {'속성명': '값' ...})
    now_tc = len(titles)
    print(f'now_tc={now_tc}')
    for t in titles:
        adult = t.find('span', {'class': "ico_comm ico_adult"})
        idx = t.text.strip()
        idx = re.sub("인\n                \n                    ","", idx)
        
        if adult is not None: # 성인물일 경우
            idx = idx[2:]
            title.append(idx)
            age[idx] = 19
            #print(f'19 {idx}')
       
        else: # 연령제한이 없는 경우   
            title.append(idx)
            age[idx] = 0
            #print(t.text)
        print(idx)
        
    # author & free & complete
    authors = bsObject.find_all('span', {'class':'txt_info'})
    for a in enumerate(authors):
        idx, name = a
        author[title[total_tc+idx]] = name.text[2:]
        free[title[total_tc+idx]] = fr_
        complete[title[total_tc+idx]] = 1
        #print(name.text[2:])
    
    # thumbnail
    thumbnails = bsObject.find_all('img', {'class': 'img_thumb'})
    for idx in range(0, now_tc):
        thumb = thumbnails[idx]['src']
        thumbnail[title[total_tc + idx]] = thumb
        #print(thumb)
     
    # 세부 주소 저장
    newlink_match = bsObject.find_all('a', {'class': 'link_wt'})
    for idx in range(0, now_tc):
        newlink = url[:-1] + newlink_match[idx]['href']
        website[title[total_tc+idx]] = newlink
        #print(newlink)
        
    # 세부 주소으로 이동 (장르, 줄거리, 평점)
    for pIdx in range(0, now_tc):
        login_check = False

        page = WebDriverWait(driver, 60).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'link_wt')))[:-10]
        print(p_, pIdx, len(page))
        driver.execute_script("arguments[0].click();", page[pIdx])
        
        
        link = driver.current_url
        
        # 성인물인 경우 로그인이 필요함
        if link[8:14] == "logins":
            login_check = True
            driver.find_element_by_css_selector('#mArticle > div.section_login > div > div > div.login_account > a.link_login.link_klogin').click()
            driver.find_element_by_name('email').send_keys(email)
            driver.find_element_by_name('password').send_keys(pw)
            driver.find_element_by_css_selector('#login-form > fieldset > div.wrap_btn > button.btn_g.btn_confirm.submit').click()

        # 다 뜰때까지 존버
        WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'txt_genre')))
        WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'txt_story')))
        WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'emph_grade')))
        
        html =  driver.page_source
        bsObject = BeautifulSoup(html, 'html.parser')

        # 장르
        genres = bsObject.find('dd', {'class': 'txt_genre'}).text.strip()
        genre[title[total_tc + pIdx]] = genres
        #print(genre)

        # 줄거리
        stories = bsObject.find('dd', {'class' : 'txt_story'}).text
        story[title[total_tc + pIdx]] = stories
        #print(story)

        # 평점
        rates = bsObject.find('em', {'class':'emph_grade'})
        rate[title[total_tc + pIdx]] = rates.text
        #print(rates.text)

        if login_check:
            driver.back()
            driver.back()
        driver.back()
        
        # 다 뜰때까지 존버
        WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'tit_wt')))
        WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'txt_info')))
        WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'img_thumb')))
        WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'link_wt')))
        
        print(driver.current_url)
    
    total_tc += now_tc
    fr_ = 1



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\gunso\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


now_tc=436
해오와 사라
경이로운 소문
나빌레라
악역의 구원자
나비는 세 번 변태한다
버닝 이펙트
왕 그리고 황제
하나밖에 안 보여!
사화
고교호구왕
하필 너야?!
의 향
그 남자의 하이힐
유색의 멜랑꼴리
보글보글챱챱
같은 방에서
특별직
용의주도 황비
내추럴 리드미칼 2
경성야상곡
괴물아기
카페 보문
묘씨맥주점
일생에 변태를 세 번 만날 확률
게뭐야
여자만화 구두
우리학교 김선생님
큐레이터
오늘도 꽐랄라라
트레이스
조우
사내 맞선
크라이시스
진심이 닿다
레드스톰
방탕일기
꽃같은 인생
우리 집에 왜 왔니
창백한 말
심봉사전
표류감옥
어름치
작약만가 : 불환곡
여의주
어둠이 걷힌 자리엔
내 일진이 이렇게 귀여울리가 없어
찬란한 액션 유치원
이태원 클라쓰
빌린 몸
저녁 같이 드실래요?
메모리스트
까락
대나무숲에서 알립니다
나는 엄마다
겨울과 봄사이
개같은 세상
돈나무
배고픈 자들의 도시
9
드림사이드
헬로 마이 안드로이드
망자의 서
달리는 여자
계약우정
스틸레인3 : 정상회담
헤븐즈 소울
ONE
해치지않아
여혜
4학년
사귀자
겟 레디 위드 미
굿바이 사돈!
아티스트
이 콤플렉스
SAVAGE
끼리끼리
승은이 망극하옵니다
안개숲
오피스텔
네 살 차이
들쥐
대디보이
체이서
껍데기
Go Bananas
에 사는 사람들
풀 뜯어먹는 소리
유사과학 탐구영역
노인의 집
밤에 사는 소녀
악연
머리 위로 부는 바람
징크스의 연인
유치원의 하루
원한 빛
과격자매단
취준생물
대치동 일타강사
시동
남은 고양이
더블랙LABEL
뿔
카운터
N번째 연애
젠틀한 야수
보통사이
단지
0.0MHz
어쩌다 발견한 7월
너도 나랑
1호선
다정한 겨울
태양의 시
그놈의 스캔들
허세남길들이기
햄스터와 그녀
먼 트리트먼트
말괄량이 길들이기
이웃잔혹사
생부
미래의 시간
빛나는 손을
그래도 되는가
포 더 프린세스
안녕, 엄마
사생
발자국이 녹기 전에
아귀
환상숲
아쿠아맨
지배
죽어도 좋아♡
우리 남매의 일상은
무능력자
우두커니
홍도
박살소녀
빈둥빈둥 환타스틱 유럽 여행기
20세 보고서
티라레
크리스마스는

http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 73 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 74 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 75 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 76 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 77 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 78 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 79 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 80 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 81 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 82 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 83 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 84 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 85 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 86 436
http:/

http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 188 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 189 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 190 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 191 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 192 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 193 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 194 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 195 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 196 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 197 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 198 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 199 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 200 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 

http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 302 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 303 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 304 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 305 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 306 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 307 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 308 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 309 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 310 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 311 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 312 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 313 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 314 436
http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
pay 

...



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\gunso\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


http://webtoon.daum.net/#sort=pay&genre=&tab=finish&day=tue
now_tc=357
단짠남녀
승리호
콜로세움
설공찬전
두번째 썸
음란마귀지만 괜찮아!
백의사자
당신도 보정해 드릴까요?
학교대표
질투유발자들
이담
요괴백정
형제가 사랑한 마녀
피치 (PITCH)
인스타러브
신랑이 사라졌다
교수님을 빚는 중
크라우드
마이셀프
시간이 머문 집
너를 만나다
화목한 가족
서천화원
치원 신드롬
옆집남자
멸종인간
조류공포증
냥냐라냥냥
구리의 구리구리 컴백
악마의 편집
미래의 너에게
퍼펙트 마인
그림자정원
2020 전남콘랩 웹툰 창작 아카데미
미래의 사진
세로토닌
질풍의 노도
랑데부
인간을 먹는 성
세터
독립만화 특별전
인생막장
노동본색
식귀
수의 계절
채널뽁스
유리의 벽
차피
아빠는 변태중
동쪽으로
거울 죽이기
Breaker.Drawing
공부하는 고양이
나와 그녀삼신과 호러맨스
조숙의 맛
트라이앵글
프롬, 스타
51번째 남자
의 새벽
마애(磨崖)
경성이 서울을 만났을 때
어떻게든 중간만 간다면
나홀로 주부
화양연화
정으로 결혼한 소꿉친구
치삼 탐정 사무소
호아전
도나스 학교괴담
이병 일학년
위대한 로맨스
옥탑빵
간이역에 겨울이 오면
다음웹툰 팬클럽 모집
대작
새벽날개
차이니즈 봉봉클럽 광저우편
곤조 스트릿 - 갱스터의 시대
애휴
헬로 브리
얌얌숲 당근길 토요요
오리지널 씬2
방랑시인 김삿갓
나는 달린다
키몽툰
작가의 사생활
리와 구
위 아 애니멀스
인랑:프리퀄
탐정 사용설명서
블레이드2 : THE RETURN OF EVIL
귀신이면 어때
스틸레인2 : 강철비
버디스
파워맨
2018 다음웹툰 능력고사
초월(初月)
컷 : 죽음의 소리
라그나로크
라디오 갤럭시
블랙 마리아
피치핏치핑크
달콩분식
어떤 거짓말
웹툰작가 살인사건
음양사
가디언즈 오브 갤럭시 :  죽음의 보석
잉어왕
장산범 : 거부할 수 없는 목소리
궁상가족
도플
용이라고 불러줘
당신의 소원은 무엇입니까
중인클럽
애프터눈 히어로즈
키스앤코리아
결혼복권
70
어쩌면, 사랑
파도

http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 75 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 76 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 77 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 78 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 79 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 80 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 81 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 82 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 83 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 84 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 85 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 86 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 87 357
http://webtoon.daum.net/#sort=free&genre=&tab=finis

http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 187 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 188 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 189 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 190 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 191 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 192 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 193 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 194 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 195 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 196 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 197 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 198 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 199 357
http://webtoon.daum.net/#sort=free&gen

http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 298 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 299 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 300 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 301 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 302 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 303 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 304 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 305 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 306 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 307 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 308 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 309 357
http://webtoon.daum.net/#sort=free&genre=&tab=finish&day=tue
free 310 357
http://webtoon.daum.net/#sort=free&gen

In [22]:
print(len(title), len(author), len(genre), len(story), len(age), len(rate), len(thumbnail), len(website))

793 793 793 793 793 793 793 793


In [23]:
import pandas as pd

dw = pd.read_csv("daum_webtoon.csv", encoding = 'euc-kr')
dw.drop(columns = ['Unnamed: 0'], inplace = True)
dw.set_index(['title'], inplace = True)
dw.head(1)

,id,author,genre,platform,story,age,rate,free,complete,thumbnail,website
title,,,,,,,,,,,
타원을 그리는 법,NaN,섬멍,"드라마, 미스터리, GL",다음 웹툰,"너무 다른 두 사람의 연애 3년, 민성의 비밀은 두 사람을 예기치 못 한 사건에 휘...",0,9.9,1,0,http://t1.daumcdn.net/webtoon/op/5604cc25c7443...,http://webtoon.daum.net/webtoon/view/drawingan...


In [24]:
for t in title:
    dw.loc[str(t)] = None

In [25]:
col = ['author', 'genre', 'story', 'age', 'rate', 'free', 'thumbnail', 'complete', 'website']
dic = [author, genre, story, age, rate, free, thumbnail, complete, website]
for i in range(0, len(dic)):
    for key, value in dic[i].items():
         dw.loc[key, col[i]] = value
dw.tail(5)

,id,author,genre,platform,story,age,rate,free,complete,thumbnail,website
title,,,,,,,,,,,
모니 앤 스토리,NaN,마이모니,"일상, 판타지, 힐링",NaN,미완성 봉제 인형 '모니'와 그의 주인 '앤'이 풀어가는 사랑이야기,0.0,9.5,1.0,1.0,http://i1.cartoon.daumcdn.net/svc/image/U03/ca...,http://webtoon.daum.net/webtoon/view/moan
천년동화,NaN,호랑,"순정, 미스터리, 리메이크",NaN,"할아버지, 할머니에게 들었던 전래동화. 그 이야기를 현대적으로 해석한다면?",0.0,9.8,1.0,1.0,http://i1.cartoon.daumcdn.net/svc/image/U03/ca...,http://webtoon.daum.net/webtoon/view/1000story
스멜스 라이크 30 스피릿,NaN,고리타,"일상, 코믹, 음악",NaN,고리타 작가 특유의 개그코드로 우울한 현대인의 자화상을 즐겁게 '편곡'한 작품,0.0,8.9,1.0,1.0,http://i1.cartoon.daumcdn.net/svc/image/U03/ca...,http://webtoon.daum.net/webtoon/view/slts
제갈주먹,NaN,청설모,"액션, 격투, 능력자",NaN,플래시 만화로 인터넷 독자들에게 사랑을 받아온 작품. 만화속세상을 통해 다시 부활했다.,0.0,9.5,1.0,1.0,http://i1.cartoon.daumcdn.net/svc/image/U03/ca...,http://webtoon.daum.net/webtoon/view/jegal
데이지1,NaN,청설모,"일상, 감성, 옴니버스",NaN,플래시 애니 '부활 이소룡 시리즈'로 많은 팬을 확보한 청설모 작가의 만화속세상 데...,0.0,9.2,1.0,1.0,http://i1.cartoon.daumcdn.net/svc/image/U03/ca...,http://webtoon.daum.net/webtoon/view/chung


In [26]:
dw['platform'] = '다음 웹툰'
dw.tail(1)

,id,author,genre,platform,story,age,rate,free,complete,thumbnail,website
title,,,,,,,,,,,
데이지1,NaN,청설모,"일상, 감성, 옴니버스",다음 웹툰,플래시 애니 '부활 이소룡 시리즈'로 많은 팬을 확보한 청설모 작가의 만화속세상 데...,0.0,9.2,1.0,1.0,http://i1.cartoon.daumcdn.net/svc/image/U03/ca...,http://webtoon.daum.net/webtoon/view/chung


In [27]:
dw = dw.reset_index().rename(columns={"index": ""})
dw = dw[['id', 'title', 'author', 'genre', 'platform', 'story', 'age', 'rate', 'free', 'complete', 'thumbnail', 'website']]

In [28]:
dw.tail(1)

,id,title,author,genre,platform,story,age,rate,free,complete,thumbnail,website
936,NaN,데이지1,청설모,"일상, 감성, 옴니버스",다음 웹툰,플래시 애니 '부활 이소룡 시리즈'로 많은 팬을 확보한 청설모 작가의 만화속세상 데...,0.0,9.2,1.0,1.0,http://i1.cartoon.daumcdn.net/svc/image/U03/ca...,http://webtoon.daum.net/webtoon/view/chung


In [29]:
dw['title'] = dw['title'].str.replace(u'\xa0', u' ')
dw['genre'] = dw['genre'].str.replace(u'\xa0', u' ')
dw['story'] = dw['story'].str.replace(u'\xa0', u' ')

In [32]:
dw.to_csv("daum.csv", mode='w', encoding = "euc-kr")